In [11]:
# ! pip install -U deepeval
# ! pip install numpy
# ! pip install transformers
# ! pip3 install torch torchvision torchaudio

In [1]:
# ! pip install sentencepiece
# ! pip install tokenizers
# ! pip install accelerate
# ! pip install bitsandbytes
# ! pip install vllm
# ! pip install fraction
# ! pip install protobuf

In [1]:
from deepeval.benchmarks import MMLU, TruthfulQA
from deepeval.benchmarks.tasks import MMLUTask, TruthfulQATask
from deepeval.benchmarks.modes import TruthfulQAMode
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM
from typing import List
import torch
import pandas as pd
import json

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {torch_device}")

/zhome/25/7/186664/dl/lib64/python3.9/site-packages/deepeval/__init__.py:53: UserWarning: You are using deepeval version 1.5.9, however version 2.0.1 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


Using device: cuda


# Model Loading

In [ ]:
blackhole_dir = #blackhole_dir

# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", cache_dir=blackhole_dir)
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", cache_dir=blackhole_dir)
bio_tokenizer = AutoTokenizer.from_pretrained("BioMistral/BioMistral-7B", cache_dir=blackhole_dir)
bio_model = AutoModelForCausalLM.from_pretrained("BioMistral/BioMistral-7B", cache_dir=blackhole_dir)

math_model = AutoModelForCausalLM.from_pretrained("meta-math/MetaMath-Mistral-7B", cache_dir=blackhole_dir)
math_tokenizer = AutoTokenizer.from_pretrained("meta-math/MetaMath-Mistral-7B", cache_dir=blackhole_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# TODO: Implement the BioMistral7B class
class BioMistral7B(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cuda" # the device to load the model onto 

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=50, do_sample=True, pad_token_id=self.tokenizer.eos_token_id, temperature=0.0001)
        decoded_tokens = self.tokenizer.batch_decode(generated_ids)[0]
        # return decoded_tokens[len(prompt)-50:]
        # return decoded_tokens
        new_tokens = decoded_tokens[len(prompt)+5:]#.strip()
        if "The answer is: " in new_tokens:
            new_tokens = new_tokens[new_tokens.find("The answer is: ")+15:]
        new_tokens = new_tokens.replace("</s>", "")
        if new_tokens.strip() == "":
            return ""
        answer_choices = [prompt[prompt.find("A. "):prompt.find("\n", prompt.find("A. "))].strip(), 
                          prompt[prompt.find("B. "):prompt.find("\n", prompt.find("B. "))].strip(), 
                            prompt[prompt.find("C. "):prompt.find("\n", prompt.find("C. "))].strip(), 
                            prompt[prompt.find("D. "):prompt.find("\n", prompt.find("D. "))].strip()]
        
        def find_answer_choice(new_tokens, answer_choices):
            if new_tokens in ["A", "B", "C", "D"]:
                return new_tokens
            for answer_choice in answer_choices:
                if new_tokens in answer_choice:
                    return answer_choice[0]
            return None
        fac = find_answer_choice(new_tokens, answer_choices)
        if fac:
            return fac

        if new_tokens[-1] == ".":
            new_tokens = new_tokens[:-1]
            fac = find_answer_choice(new_tokens, answer_choices)
            if fac:
                return fac
        if "\\text{" in new_tokens:
            new_tokens = new_tokens.replace("\\text{", "")
            new_tokens = new_tokens[:-1]
            fac = find_answer_choice(new_tokens, answer_choices)
            if fac:
                return fac
        if new_tokens[0] == "(" or new_tokens[0] == "[":
            new_tokens = new_tokens[1:-1]
            fac = find_answer_choice(new_tokens, answer_choices)
            if fac:
                return fac
        return new_tokens
        new_tokens = new_tokens[new_tokens.find("{"):new_tokens.rfind("}")+1]
        try:
            new_tokens_json = json.loads(new_tokens, strict=False)
            return new_tokens_json['Answer_choice']
        except:
            new_tokens = new_tokens.replace("{", "").replace("}", "")
            new_tokens = "{" + new_tokens + "}"
            print(new_tokens)
            return new_tokens
        return decoded_tokens[len(prompt):]
        return decoded_tokens[len(prompt)+5:]
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    # This is optional.
    def batch_generate(self, promtps: List[str]) -> List[str]:
        model = self.load_model()
        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer(promtps, return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)

    def get_model_name(self):
        return "BioMistral 7B"

biomistral_7b = BioMistral7B(model=bio_model, tokenizer=bio_tokenizer)

In [5]:
class MetaMath7B(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cuda" # the device to load the model onto 

        # prompt = "[INST] " + prompt + " [/INST]"
        # prompt = prompt + "\nYou will only respond with a JSON object with the keys Explanation and Answer Choice, where Explanation is a string and Answer Choice is either 'A', 'B', 'C' or 'D'."
        # prompt = """Please answer the following question strictly in JSON format. Your response must contain only a JSON object 
        # with the keys 'Explanation' and 'Answer_choice'. The 'Answer_choice' should be one of four options: 'A', 'B', 'C' or 'D'. 
        # No extra text, tokens or explanation outside the JSON format is allowed.\n
        # Examples on how to answer questions:\n
        # What color is the sky?\nA. green\nB. blue \nC. red\nD. orange\nAnswer: 
        # {"Explanation": "The color of the sky depends on the time of day, the weather, and the location. In general, the sky is blue during the day, but it can also be other colors such as gray or black during certain weather conditions. Therefore, the answer cannot be determined with the given information.",
        # "Answer_choice": "D"}
        
        # Answer the following question with the json format and nothing else then stop generating:\n""" + prompt[:-57]

        # prompt = """
        # The correct answer to the following question is formatted as a JSON object with the keys "Explanation" and "Answer_Choice". The "Explanation" key should contain a string explaining the answer, and the "Answer_Choice" key should contain the letter of the correct answer, either A, B, C or D. For example, if the correct answer is "A", the JSON object should look like this: {"Explanation": "This is the explanation.", "Answer_Choice": "A"}. I want you to only print out the value of the key "Answer_Choice". The question is as follows:\n 
        # """ + prompt[:-57]

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=50, do_sample=True, pad_token_id=self.tokenizer.eos_token_id, temperature=0.0001)
        decoded_tokens = self.tokenizer.batch_decode(generated_ids)[0]
        # return decoded_tokens[len(prompt)-50:]
        # return decoded_tokens
        new_tokens = decoded_tokens[len(prompt)+5:]#.strip()
        if "The answer is: " in new_tokens:
            new_tokens = new_tokens[new_tokens.find("The answer is: ")+15:]
        new_tokens = new_tokens.replace("</s>", "")
        if new_tokens.strip() == "":
            return ""
        answer_choices = [prompt[prompt.find("A. "):prompt.find("\n", prompt.find("A. "))].strip(), 
                          prompt[prompt.find("B. "):prompt.find("\n", prompt.find("B. "))].strip(), 
                            prompt[prompt.find("C. "):prompt.find("\n", prompt.find("C. "))].strip(), 
                            prompt[prompt.find("D. "):prompt.find("\n", prompt.find("D. "))].strip()]
        
        def find_answer_choice(new_tokens, answer_choices):
            if new_tokens in ["A", "B", "C", "D"]:
                return new_tokens
            for answer_choice in answer_choices:
                if new_tokens in answer_choice:
                    return answer_choice[0]
            return None
        fac = find_answer_choice(new_tokens, answer_choices)
        if fac:
            return fac

        if new_tokens[-1] == ".":
            new_tokens = new_tokens[:-1]
            fac = find_answer_choice(new_tokens, answer_choices)
            if fac:
                return fac
        if "\\text{" in new_tokens:
            new_tokens = new_tokens.replace("\\text{", "")
            new_tokens = new_tokens[:-1]
            fac = find_answer_choice(new_tokens, answer_choices)
            if fac:
                return fac
        if new_tokens[0] == "(" or new_tokens[0] == "[":
            new_tokens = new_tokens[1:-1]
            fac = find_answer_choice(new_tokens, answer_choices)
            if fac:
                return fac
        return new_tokens
        new_tokens = new_tokens[new_tokens.find("{"):new_tokens.rfind("}")+1]
        try:
            new_tokens_json = json.loads(new_tokens, strict=False)
            return new_tokens_json['Answer_choice']
        except:
            new_tokens = new_tokens.replace("{", "").replace("}", "")
            new_tokens = "{" + new_tokens + "}"
            print(new_tokens)
            return new_tokens
        return decoded_tokens[len(prompt):]
        return decoded_tokens[len(prompt)+5:]
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    # This is optional.
    def batch_generate(self, promtps: List[str]) -> List[str]:
        model = self.load_model()
        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer(promtps, return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)

    def get_model_name(self):
        return "MetaMath Mistral 7B"

metamath_7b = MetaMath7B(model=math_model, tokenizer=math_tokenizer)

In [ ]:
# MetaMath
# We want to evaluate the model by calculating its overall accuracy (this is bad, at around ~12%),
# as well as its accuracy when it answers in the correct format (A, B, C or D) - which is hopefully better.

In [242]:
print(metamath_7b.generate("What is 5 minus 2?\nA. 3\nB. 5\nC. 25\nD. 50\nAnswer:\n\nOutput 'A', 'B', 'C', or 'D'. Full answer not needed."))

A


In [243]:
print(metamath_7b.generate("The following are multiple choice questions (with answers) about high school biology.\n\nWhich of the following is not a way to form recombinant DNA?\nA. Translation\nB. Conjugation\nC. Specialized transduction\nD. Transformation\nAnswer:\n\nOutput 'A', 'B', 'C', or 'D'. Full answer not needed."))

B


In [244]:
print(metamath_7b.generate("If a metamath_7bn P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\nA. (0, – 3)\nB. (4, 1)\nC. (2, 2)\nD. (– 4, –2)\nAnswer:\n\nOutput 'A', 'B', 'C', or 'D'. Full answer not needed."))

B


---

# Model Evaluation

In [ ]:
mm_tasks_math = [MMLUTask.HIGH_SCHOOL_MATHEMATICS]
mm_tasks_bio = [MMLUTask.HIGH_SCHOOL_BIOLOGY]
tqa_tasks = [TruthfulQATask.SCIENCE]
tqa_mode = TruthfulQAMode.MC1 # Use MC1 as a benchmark for pinpoint accuracy and MC2 for depth of understanding.

# Define benchmark with specific tasks and shots
math_benchmark = MMLU(
    tasks=mm_tasks_math,
    n_shots=0
)

bio_benchmark = MMLU(
    tasks=mm_tasks_bio,
    n_shots=0
)

TQABenchmark = TruthfulQA(
    tasks=tqa_tasks,
    mode=tqa_mode
)

In [ ]:
# Replace 'mistral_7b' with your own custom model
math_benchmark.evaluate(model=metamath_7b)
math_results = math_benchmark.predictions
print('-'*50)
print(f'Overall score for {metamath_7b.get_model_name()}: {math_benchmark.overall_score}')

Processing high_school_mathematics: 100%|█████| 270/270 [02:12<00:00,  2.04it/s]

MMLU Task Accuracy (task=high_school_mathematics): 0.1259259259259259
Overall MMLU Accuracy: 0.1259259259259259
0.1259259259259259


In [ ]:
bio_benchmark.evaluate(model=biomistral_7b)
bio_results = bio_benchmark.predictions
print('-'*50)
print(f'Overall score for {biomistral_7b.get_model_name()}: {bio_benchmark.overall_score}')

In [ ]:
results_corrform = math_results
# del_index = results_corrform[(results_corrform['Prediction']) != 'A' | (results_corrform['Prediction']) != 'B' | (results_corrform['Prediction']) != 'C' | (results_corrform['Prediction']) != 'D'].index


In [275]:
results_corrform = results_corrform[(results_corrform['Prediction'] == 'A') | (results_corrform['Prediction'] == 'B') | (results_corrform['Prediction'] == 'C') | (results_corrform['Prediction'] == 'D')]
correct = 0
for index, row in results_corrform.iterrows():
    if row['Correct'] == 1:
        correct += 1
print(correct/len(results_corrform))

0.288135593220339


In [ ]:
pd.set_option("display.max_rows", None)
math_results

,Task,Input,Prediction,Correct


In [ ]:
for i in range(20):
    print(f'Input {i+1}:')
    print('=**********=')
    print(math_benchmark.predictions['Input'][i])
    print('=**********=')
    print(f'Prediction {i+1}:')
    print('++++++++++')
    # print(results['Prediction'][i][results['Prediction'][i].find("\n\nOutput 'A', 'B', 'C', or 'D'. Full answer not needed.")+55:])
    print(math_results['Prediction'][i])
    print('++++++++++')
    print(math_results['Correct'][i])
    print('-'*50)

Input 1:
=**********=
If a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is
A. (0, – 3)
B. (4, 1)
C. (2, 2)
D. (– 4, –2)
Answer:
=**********=
Prediction 1:
++++++++++
<s> The following are multiple choice questions (with answers) about high school mathematics.

Joe was in charge of lights for a dance. The red light blinks every two seconds, the yellow light every three seconds, and the blue light every five seconds. If we include the very beginning and very end of the dance, how many times during a seven minute dance will all the lights come on at the same time? (Assume that all three lights blink simultaneously at the very beginning of the dance.)
A. 3
B. 15
C. 6
D. 5
Answer: B

Five thousand dollars compounded annually at an $x\%$ interest rate takes six years to double. At the same interest rate, how many years will it take $\$300$ to grow to $\$9600$?
A. 12
B